In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
import pprint
from sklearn.preprocessing import StandardScaler
%matplotlib inline

from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import r2_score, roc_auc_score, roc_curve, classification_report
import xgboost as xgb
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold

DATASET1 = 'march madness past data.csv'
DATASET2 = 'march madness this year.csv'

In [45]:
df = pd.read_csv(DATASET1)
df2 = pd.read_csv(DATASET2)
print(df.head(10))

   YEAR  SEED  target  K TEMPO  K TEMPO RANK   KADJ T  KADJ T RANK    K OFF  \
0  2023     1       4  72.8260            12  72.7469            4  110.862   
1  2023     2       6  72.8989            10  72.3118            9  113.208   
2  2023    11       6  69.1548           121  68.4035          131  100.983   
3  2023     8       4  70.1147            73  69.6808           54  105.743   
4  2023     9       5  68.6925           145  67.6112          165  105.387   
5  2023     3       5  67.8723           185  66.5510          217  113.141   
6  2023    10       6  67.0127           229  66.5002          219  105.781   
7  2023    15       6  68.0072           179  67.8783          156  114.974   
8  2023    12       6  70.8386            44  70.9567           29  112.460   
9  2023     4       0  68.5693           154  66.6901          210  114.228   

   KO RANK   KADJ O  ...  EFF HGT RANK  EXP RANK  TALENT RANK  FT% RANK  \
0       32  116.097  ...            10       353       

In [46]:
df.isnull().sum()

YEAR              0
SEED              0
target            0
K TEMPO           0
K TEMPO RANK      0
                 ..
PPPO RANK         0
PPPD RANK         0
ELITE SOS RANK    0
CONF_encoded      0
TEAM_encoded      0
Length: 98, dtype: int64

In [47]:
# Split dataset into X and Y
X_train = df.iloc[:, df.columns != 'target']
y_train = df['target']
X_test = df2.iloc[:, df2.columns != 'target']

In [48]:
#Creating an XGBoost classifier
model = xgb.XGBClassifier()

#Training the model on the training data
model.fit(X_train, y_train)

#Making predictions on the test set
y_pred = model.predict(X_test)

In [49]:
print(y_pred)

[6 4 5 4 6 6 5 6 6 6 6 5 0 4 6 6 4 6 6 6 4 7 6 4 6 6 4 5 4 4 6 6 3 6 5 6 6
 6 5 6 5 4 6 6 6 5 4 5 6 6 5 5 6 6 6 5 6 5 6 6 5 6 6 6 6 6 4 6]


In [50]:
df2.insert(0, "Prediction", y_pred, True)

In [52]:
for ind in df2.index:
    if df2['Prediction'][ind] == 0:
        df2['Prediction'][ind] = 'CHAMP'
    if df2['Prediction'][ind] == 1:
        df2['Prediction'][ind] = 'QTR'
    if df2['Prediction'][ind] == 2:
        df2['Prediction'][ind] = 'SEMI'
    if df2['Prediction'][ind] == 3:
        df2['Prediction'][ind] = 'QTR'
    if df2['Prediction'][ind] == 4:
        df2['Prediction'][ind] = 'R16'
    if df2['Prediction'][ind] == 5:
        df2['Prediction'][ind] = 'R32'
    if df2['Prediction'][ind] == 6:
        df2['Prediction'][ind] = 'R64'
    if df2['Prediction'][ind] == 7:
        df2['Prediction'][ind] = 'FIRST4'
        
df2.head()

<ipython-input-52-50e56f7f215d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Prediction'][ind] = 'R64'
C:\Users\yudai\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-52-50e56f7f215d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Prediction'][ind] = 'R16'
<ipython-input-52-50e56f7f215d>:13: SettingWi

,Prediction,YEAR,SEED,target,K TEMPO,K TEMPO RANK,KADJ T,KADJ T RANK,K OFF,KO RANK,...,EFF HGT RANK,EXP RANK,TALENT RANK,FT% RANK,OP FT% RANK,PPPO RANK,PPPD RANK,ELITE SOS RANK,CONF_encoded,TEAM_encoded
0,R64,2024,14,0,66.7747,274,65.8933,268,107.841,122,...,199,19,176,164,47,122,51,249,16,1
1,R16,2024,4,0,74.1625,7,72.6461,9,121.712,2,...,8,156,106,10,314,2,263,7,27,2
2,R32,2024,2,0,73.3760,11,71.8379,16,117.653,10,...,37,196,7,195,134,8,14,47,23,7
3,R16,2024,4,0,70.9629,54,69.7887,58,117.364,14,...,76,127,69,59,284,12,8,69,27,11
4,R64,2024,3,0,66.8428,270,65.6032,281,117.262,15,...,22,304,34,97,254,15,155,1,6,14


In [53]:
df2.to_csv('march madness this year.csv', encoding='utf-8', index=False)